## Data loading

In [11]:
import numpy as np
import pandas as pd
import os
import sys
from math import frexp, ldexp
import math
import time
from condTE_utils import CondTEUtils as CTU

In [12]:
# load data
datapath = '../Firearms'
data_original = pd.read_excel(os.path.join(datapath,'summary_data.xlsx'))
BC = data_original['Background checks (48 states, processed)'].values
MS = data_original['Mass shooting'].values
MO = data_original.values[:,2:5]


In [13]:
# symbolic
XBC = (np.diff(BC,axis=0) > 0) * 1
XMS = (MS[:-1] > 0) * 1
XMO = (np.diff(MO,axis=0) > 0) * 1
Nt = len(XBC)

# Experiment

In [23]:
nsurr = 50000

In [24]:
# conditional transfer entropies betweeen every pair (MO_laws)
LabelsCond_laws = np.array(['MS->BC/MO_laws','MO_laws->BC/MS','BC->MS/MO_laws',
                        'MO_laws->MS/BC','BC->MO_laws/MS','MS->MO_laws/BC'])

conTEResults_values_laws = np.array([CTU.cal_transfer_entropy(XBC,XMS,XMO[:,0]),
                                     CTU.cal_transfer_entropy(XBC,XMO[:,0],XMS),
                                     CTU.cal_transfer_entropy(XMS,XBC,XMO[:,0]),
                                     CTU.cal_transfer_entropy(XMS,XMO[:,0],XBC),
                                     CTU.cal_transfer_entropy(XMO[:,0],XBC,XMS),
                                     CTU.cal_transfer_entropy(XMO[:,0],XMS,XBC)])

conTEResults_laws = {}
conTEResults_laws = {label : conTEResults_values_laws[i] for i,label in enumerate(LabelsCond_laws)}
conTEResults_laws

{'MS->BC/MO_laws': 0.013431993682661236,
 'MO_laws->BC/MS': 0.038857029521437436,
 'BC->MS/MO_laws': 0.018444077159448735,
 'MO_laws->MS/BC': 0.011364806635881575,
 'BC->MO_laws/MS': 0.002905192435187498,
 'MS->MO_laws/BC': 0.008587731438766788}

In [16]:
ndarr = np.stack([XBC], axis=1)
sample_space = CTU.get_sample_space(ndarr)
str_arr = np.array([''.join(row.astype(str)) for row in ndarr])
positions = [str_arr == sample for sample in sample_space]
XBC[positions[0]]

AttributeError: type object 'CondTEUtils' has no attribute 'get_sample_space'

In [25]:
# P-value test((MO_laws))
sur_conTEresults_laws_values = np.array([CTU.surrogate_cond_transfer_entropy(XBC,XMS,XMO[:,0],nsurr),
                                         CTU.surrogate_cond_transfer_entropy(XBC,XMO[:,0],XMS,nsurr),
                                         CTU.surrogate_cond_transfer_entropy(XMS,XBC,XMO[:,0],nsurr),
                                         CTU.surrogate_cond_transfer_entropy(XMS,XMO[:,0],XBC,nsurr),
                                         CTU.surrogate_cond_transfer_entropy(XMO[:,0],XBC,XMS,nsurr),
                                         CTU.surrogate_cond_transfer_entropy(XMO[:,0],XMS,XBC,nsurr)])

sur_conTEresults_laws = {}
sur_conTEresults_laws = {label: sur_conTEresults_laws_values[i] for i,label in enumerate(LabelsCond_laws)}


p_values_laws = {}
ninetyfive_tab_laws = {}

for i, label in enumerate(LabelsCond_laws):
    aux_count_sum = np.sum(sur_conTEresults_laws[label] > conTEResults_laws[label])
    p_values_laws[label] = (aux_count_sum + 1) / (nsurr + 1)
    ninetyfive_tab_laws[label] = np.quantile(sur_conTEresults_laws[label] , 0.95)
p_values_laws


KeyboardInterrupt: 

In [ ]:
# conditional transfer entropies betweeen every pair (MO_shooting)
LabelsCond_shooting = np.array(['MS->BC/MO_shooting','MO_shooting->BC/MS','BC->MS/MO_shooting',
                        'MO_shooting->MS/BC','BC->MO_shooting/MS','MS->MO_shooting/BC'])

conTEResults_values_shooting = np.array([CTU.cal_transfer_entropy(XBC,XMS,XMO[:,2]),
                                         CTU.cal_transfer_entropy(XBC,XMO[:,2],XMS),
                                         CTU.cal_transfer_entropy(XMS,XBC,XMO[:,2]), 
                                         CTU.cal_transfer_entropy(XMS,XMO[:,2],XBC),
                                         CTU.cal_transfer_entropy(XMO[:,2],XBC,XMS), 
                                         CTU.cal_transfer_entropy(XMO[:,2],XMS,XBC)])

conTEResults_shooting = {}
conTEResults_shooting = {label : conTEResults_values_shooting[i] for i,label in enumerate(LabelsCond_shooting)}
conTEResults_shooting

In [ ]:
# P-value test((MO_shooting)
sur_conTEresults_shooting_values = np.array([CTU.surrogate_cond_transfer_entropy(XBC,XMS,XMO[:,2],nsurr),
                                             CTU.surrogate_cond_transfer_entropy(XBC,XMO[:,2],XMS,nsurr),
                                             CTU.surrogate_cond_transfer_entropy(XMS,XBC,XMO[:,2],nsurr),
                                             CTU.surrogate_cond_transfer_entropy(XMS,XMO[:,2],XBC,nsurr),
                                             CTU.surrogate_cond_transfer_entropy(XMO[:,2],XBC,XMS,nsurr),
                                             CTU.surrogate_cond_transfer_entropy(XMO[:,2],XMS,XBC,nsurr)])

sur_conTEresults_shooting = {}
sur_conTEresults_shooting = {label: sur_conTEresults_shooting_values[i] for i,label in enumerate(LabelsCond_shooting)}


p_values_shooting = {}
ninetyfive_tab_shooting = {}

for i, label in enumerate(LabelsCond_shooting):
    aux_count_sum = np.sum(sur_conTEresults_shooting[label] > conTEResults_shooting[label])
    p_values_shooting[label] = (aux_count_sum + 1) / (nsurr + 1)
    ninetyfive_tab_shooting[label] = np.quantile(sur_conTEresults_shooting[label] , 0.95)
p_values_shooting

In [ ]:
# conditional transfer entropies betweeen every pair (MO_unemployment)
LabelsCond_unemployment = np.array(['MS->BC/MO_unemployment','MO_unemployment>BC/MS','BC->MS/MO_unemployment',
                        'MO_unemployment->MS/BC','BC->MO_unemployment/MS','MS->MO_unemployment/BC'])

conTEResults_values_unemployment = np.array([CTU.cal_transfer_entropy(XBC,XMS,XMO[:,1]), 
                                             CTU.cal_transfer_entropy(XBC,XMO[:,1],XMS),
                                             CTU.cal_transfer_entropy(XMS,XBC,XMO[:,1]), 
                                             CTU.cal_transfer_entropy(XMS,XMO[:,1],XBC),
                                             CTU.cal_transfer_entropy(XMO[:,1],XBC,XMS),
                                             CTU.cal_transfer_entropy(XMO[:,1],XMS,XBC)])

conTEResults_unemployment = {}
conTEResults_unemployment = {label : conTEResults_values_unemployment[i] for i,label in enumerate(LabelsCond_unemployment)}


In [ ]:
# P-value test((MO_unemployment)
sur_conTEresults_unemployment_values = np.array([CTU.surrogate_cond_transfer_entropy(XBC,XMS,XMO[:,1],nsurr),
                                                 CTU.surrogate_cond_transfer_entropy(XBC,XMO[:,1],XMS,nsurr),
                                                 CTU.surrogate_cond_transfer_entropy(XMS,XBC,XMO[:,1],nsurr),
                                                 CTU.surrogate_cond_transfer_entropy(XMS,XMO[:,1],XBC,nsurr),
                                                 CTU.surrogate_cond_transfer_entropy(XMO[:,1],XBC,XMS,nsurr),
                                                 CTU.surrogate_cond_transfer_entropy(XMO[:,1],XMS,XBC,nsurr)])

sur_conTEresults_unemployment = {}
sur_conTEresults_unemployment = {label: sur_conTEresults_unemployment_values[i] for i,label in enumerate(LabelsCond_unemployment)}


p_values_unemployment = {}
ninetyfive_tab_unemployment = {}

for i, label in enumerate(LabelsCond_unemployment):
    aux_count_sum = np.sum(sur_conTEresults_unemployment[label] > conTEResults_unemployment[label])
    p_values_unemployment[label] = (aux_count_sum + 1) / (nsurr + 1)
    ninetyfive_tab_unemployment[label] = np.quantile(sur_conTEresults_unemployment[label] , 0.95)
p_values_unemployment